In [1]:
import os
import sys
import numpy as np
from omegaconf import DictConfig, OmegaConf



In [2]:
def load_and_get_results(sim, rag, primitive, optim, fd):

    # GET THE NR OF ATTEMPTS
    results_list = OmegaConf.load(fd)
    # Create or pandas or iterate....
    print(f"\n ==Analysing sim={sim} rag={rag} primitive={primitive} optimiser={optim} length={len(results_list)}")

    # for key, value in results_dict.items():
    target_list= []
    params_list = []
    
    for i in range(len(results_list)):
        loss = float(results_list[i]['target'])
        values_dict = results_list[i]['params']
        values_list = []
        for key, val in values_dict.items():
            values_list.append(val)
        
        target_list.append(loss)
        params_list.append(np.array(values_list))
        
    targets_arr = np.array(target_list)
    params_arr = np.array(params_list)
    min_idx = np.argmin(targets_arr)
    min_value = targets_arr[min_idx]
    min_params = params_arr[min_idx]
    
    if len(results_list) < 100:
        print(f"Length of list = {len(results_list)}, min target={min_value}")
    
    if len(results_list) < 10:
        print(f"Prompting the list <10")
        for i in range(len(results_list)):
            print(f"{results_list[i]}")
    
    return targets_arr, params_arr, min_value, min_params



In [3]:
resuts_dir = "/home/mulerod1/projects/dynamic_dual_manip/sys_identif_29/"


def get_targets_and_params(requested_optimiser):
    
    dynamic_sim_results = {
        'mujoco3': {
            'cotton_rag_0': {},
            'chequered_rag_0': {},
            'linen_rag_0': {},
        },
        'bullet': {
            'cotton_rag_0': {},
            'chequered_rag_0': {},
            'linen_rag_0': {},
        },
        'bullet_w_scale': {
            'cotton_rag_0': {},
            'chequered_rag_0': {},
            'linen_rag_0': {},
        },
        'softgym': {
            'cotton_rag_0': {},
            'chequered_rag_0': {},
            'linen_rag_0': {},
        },
        'sofa': {
            'cotton_rag_0': {},
            'chequered_rag_0': {},
            'linen_rag_0': {},
        },
    }
    quasi_static_sim_results = {
        'mujoco3': {
            'cotton_rag_0': {},
            'chequered_rag_0': {},
            'linen_rag_0': {},
        },
        'bullet': {
            'cotton_rag_0': {},
            'chequered_rag_0': {},
            'linen_rag_0': {},
        },
        'bullet_w_scale': {
            'cotton_rag_0': {},
            'chequered_rag_0': {},
            'linen_rag_0': {},
        },
        'softgym': {
            'cotton_rag_0': {},
            'chequered_rag_0': {},
            'linen_rag_0': {},
        },
        'sofa': {
            'cotton_rag_0': {},
            'chequered_rag_0': {},
            'linen_rag_0': {},
        },
    }

    for server in os.listdir(resuts_dir):
        servers_dir = f"{resuts_dir}/{server}/multirun/"
        for optim in os.listdir(servers_dir):
            optim_dir = servers_dir+optim
            if requested_optimiser in optim:
                pass
            else:
                continue
            for sim_engine in os.listdir(optim_dir):
                
                if "softgym" in sim_engine:
                    continue
                elif "old" in sim_engine:
                    continue

                sim_dir = optim_dir+"/"+sim_engine
                for rag_type in os.listdir(sim_dir):
                    rag_dir = sim_dir + "/" + rag_type
                    for primitive in os.listdir(rag_dir):
                        primitive_dir = rag_dir + "/" + primitive
                        for fd in os.listdir(primitive_dir):
                            if "log" in fd:
                                results_file=os.path.join(primitive_dir, fd)
                                # print(f"looking at file={results_file}")
                                (targets_arr, params_arr, 
                                 min_value, min_params) = load_and_get_results(sim=sim_engine, fd=results_file,
                                                                               rag=rag_type, primitive=primitive,
                                                                               optim=requested_optimiser)
                                
                                if "dynamic" in primitive:
                                    dynamic_sim_results[sim_engine][rag_type]['target'] = min_value
                                    dynamic_sim_results[sim_engine][rag_type]['params'] = min_params
                                else:
                                    quasi_static_sim_results[sim_engine][rag_type]['target'] = min_value
                                    quasi_static_sim_results[sim_engine][rag_type]['params'] = min_params
    return quasi_static_sim_results, dynamic_sim_results

In [4]:

bo_quasi, bo_dyn = get_targets_and_params(requested_optimiser="BO")

cmaes_quasi, cmaes_dyn = get_targets_and_params(requested_optimiser="Cma")

# requested_optim = "Cma"


 ==Analysing sim=sofa rag=cotton_rag_0 primitive=quasi_static optimiser=BO length=15
Length of list = 15, min target=0.08676550537347794

 ==Analysing sim=sofa rag=cotton_rag_0 primitive=dynamic optimiser=BO length=15
Length of list = 15, min target=0.0856267437338829

 ==Analysing sim=sofa rag=chequered_rag_0 primitive=quasi_static optimiser=BO length=105

 ==Analysing sim=sofa rag=chequered_rag_0 primitive=dynamic optimiser=BO length=15
Length of list = 15, min target=0.08535206317901611

 ==Analysing sim=sofa rag=linen_rag_0 primitive=quasi_static optimiser=BO length=25
Length of list = 25, min target=0.08195144683122635

 ==Analysing sim=sofa rag=linen_rag_0 primitive=dynamic optimiser=BO length=95
Length of list = 95, min target=0.06339991092681885

 ==Analysing sim=bullet rag=cotton_rag_0 primitive=dynamic optimiser=BO length=145

 ==Analysing sim=bullet rag=chequered_rag_0 primitive=dynamic optimiser=BO length=175

 ==Analysing sim=bullet rag=linen_rag_0 primitive=dynamic optim

In [6]:
import pandas as pd

In [7]:
a = pd.DataFrame.from_dict(bo_quasi)
b = pd.DataFrame.from_dict(bo_dyn)

c = pd.DataFrame.from_dict(cmaes_quasi)
d = pd.DataFrame.from_dict(cmaes_dyn)

print(f"\n == Quasi-static ===")
print(f"1) BO")
print(a)
print(f"\n2) CMA-ES")
print(c)

print(f"\n == Dynamic ===")
print(f"1) BO")
print(b)
print(f"\n2) CMA-ES")
print(d)



 == Quasi-static ===
1) BO
                                                           mujoco3 bullet  \
cotton_rag_0     {'target': 0.0697827860713005, 'params': [0.69...     {}   
chequered_rag_0  {'target': 0.05338621139526367, 'params': [0.1...     {}   
linen_rag_0      {'target': 0.06920301914215088, 'params': [0.4...     {}   

                                                    bullet_w_scale softgym  \
cotton_rag_0                                                    {}      {}   
chequered_rag_0  {'target': 0.08450822532176971, 'params': [37....      {}   
linen_rag_0                                                     {}      {}   

                                                              sofa  
cotton_rag_0     {'target': 0.08676550537347794, 'params': [449...  
chequered_rag_0  {'target': 0.07188375294208527, 'params': [310...  
linen_rag_0      {'target': 0.08195144683122635, 'params': [223...  

2) CMA-ES
                                                           mujo

In [58]:
b = pd.DataFrame.from_dict(bo_dyn)

d = pd.DataFrame.from_dict(cmaes_dyn)

print(f"\n == Dynamic ===")
print(f"1) BO")
print(b)
print(f"\n2) CMA-ES")
print(d)



 == Dynamic ===
1) BO
                                                           mujoco3  \
cotton_rag_0     {'target': 0.07147571444511414, 'params': [0.9...   
chequered_rag_0  {'target': 0.055277302861213684, 'params': [1....   
linen_rag_0      {'target': 0.07604530453681946, 'params': [0.6...   

                                                            bullet  \
cotton_rag_0     {'target': 0.1590772271156311, 'params': [0.00...   
chequered_rag_0  {'target': 0.2003229260444641, 'params': [0.00...   
linen_rag_0      {'target': 0.22234511375427246, 'params': [10....   

                                                    bullet_w_scale softgym  \
cotton_rag_0     {'target': 0.21476605534553528, 'params': [39....      {}   
chequered_rag_0                                                 {}      {}   
linen_rag_0                                                     {}      {}   

                                                              sofa  
cotton_rag_0     {'target': 0.085

In [60]:


def print_params(sim, params_dict, primitive):
    for key, items in params_dict[sim].items():
        sim_params = items['params']
        print(f"\n SIM==={sim}")
        print(f'Sim={sim} rag={key} params={sim_params}')
        for i in range(3):
            print(f"params_{primitive}_{key[:-2]}_{i}:")
            if sim == 'sofa':
                print(f"  youngmodulus: {sim_params[0]}")
                print(f"  poissonratio: {sim_params[1]}")
                print(f"  damping: {sim_params[2]}")
                print(f"  stiffness: {sim_params[3]}")
                print(f"  vertex_mass: {sim_params[4]}")
                # print(f"\n  scale={sim_params[5]}")
            elif sim == 'mujoco3':
                print(f"  edge_damping: {sim_params[0]}")
                print(f"  mass: {sim_params[1]}")
                print(f"  plugin_config_poisson_value: {sim_params[2]}")
                print(f"  plugin_config_thickness_value: {sim_params[3]}")
                print(f"  plugin_config_young_value: {sim_params[4]}")
            else:  # BULLET
                print(f"  deform_bending_stiffness: {sim_params[0]}")
                print(f"  deform_damping_stiffness: {sim_params[1]}")
                print(f"  deform_elastic_stiffness: {sim_params[2]}")
                print(f"  deform_friction_coeff: {sim_params[3]}")
                print(f"  mass: {sim_params[4]}")
                if 'scale' in sim:
                    print(f"  scale: {sim_params[5]}")
                else:
                    print(f"  scale:  1.0")

In [43]:
print("QUASI-STATIC")
print_params(sim='mujoco3', params_dict=bo_quasi, primitive='quasi_static')
print_params(sim='sofa', params_dict=cmaes_quasi, primitive='quasi_static')

print("\n DYNAMIC")
print_params(sim='mujoco3', params_dict=bo_dyn, primitive='dynamic')
print_params(sim='sofa', params_dict=cmaes_dyn, primitive='dynamic')

QUASI-STATIC

 SIM===mujoco3
Sim=mujoco3 rag=cotton_rag_0 params=[6.92710861e-01 1.06445973e+01 4.14071226e-01 1.23847812e-03
 1.08469054e+04]
params_quasi_static_cotton_rag_0:
  edge_damping: 0.6927108607000713
  mass: 10.644597310715174
  plugin_config_poisson_value: 0.4140712258638586
  plugin_config_thickness_value: 0.0012384781246429707
  plugin_config_young_value: 10846.905447469928
params_quasi_static_cotton_rag_1:
  edge_damping: 0.6927108607000713
  mass: 10.644597310715174
  plugin_config_poisson_value: 0.4140712258638586
  plugin_config_thickness_value: 0.0012384781246429707
  plugin_config_young_value: 10846.905447469928
params_quasi_static_cotton_rag_2:
  edge_damping: 0.6927108607000713
  mass: 10.644597310715174
  plugin_config_poisson_value: 0.4140712258638586
  plugin_config_thickness_value: 0.0012384781246429707
  plugin_config_young_value: 10846.905447469928

 SIM===mujoco3
Sim=mujoco3 rag=chequered_rag_0 params=[1.89584850e-01 1.16809632e+01 4.83287224e-01 9.0420394

In [46]:
print("QUASI-STATIC")
print_params(sim='bullet', params_dict=cmaes_quasi, primitive='quasi_static')
print("\n==== BO - QUASI-STATIC ===")
print_params(sim='bullet', params_dict=bo_quasi, primitive='quasi_static')

print("\n DYNAMIC")
print_params(sim='bullet', params_dict=cmaes_dyn, primitive='dynamic')
# print_params(sim='bullet_w_scale', params_dict=cmaes_dyn, primitive='dynamic')

QUASI-STATIC

 SIM===bullet
Sim=bullet rag=cotton_rag_0 params=[2.31000000e+01 1.09361321e-02 1.57549871e+01 1.05734672e+01
 5.12433079e+00]
params_quasi_static_cotton_rag_0:
  deform_bending_stiffness: 23.1
  deform_damping_stiffness: 0.010936132113290697
  deform_elastic_stiffness: 15.754987119332865
  deform_friction_coeff: 10.573467227076312
  mass: 5.124330792242306
params_quasi_static_cotton_rag_1:
  deform_bending_stiffness: 23.1
  deform_damping_stiffness: 0.010936132113290697
  deform_elastic_stiffness: 15.754987119332865
  deform_friction_coeff: 10.573467227076312
  mass: 5.124330792242306
params_quasi_static_cotton_rag_2:
  deform_bending_stiffness: 23.1
  deform_damping_stiffness: 0.010936132113290697
  deform_elastic_stiffness: 15.754987119332865
  deform_friction_coeff: 10.573467227076312
  mass: 5.124330792242306

 SIM===bullet
Sim=bullet rag=chequered_rag_0 params=[2.81000000e+01 2.72340047e-02 1.07036979e+01 7.41228797e+00
 5.05020557e+00]
params_quasi_static_chequered

In [62]:
print("\n DYNAMIC")
# print_params(sim='bullet', params_dict=cmaes_dyn, primitive='dynamic')
print_params(sim='bullet_w_scale', params_dict=cmaes_dyn, primitive='dynamic')


 DYNAMIC

 SIM===bullet_w_scale
Sim=bullet_w_scale rag=cotton_rag_0 params=[26.1         2.3954297  11.11399114  8.88510494  9.05070882 49.94753606]
params_dynamic_cotton_rag_0:
  deform_bending_stiffness: 26.1
  deform_damping_stiffness: 2.3954296985162875
  deform_elastic_stiffness: 11.11399113550867
  deform_friction_coeff: 8.885104935403207
  mass: 9.050708824606582
  scale: 49.94753606362635
params_dynamic_cotton_rag_1:
  deform_bending_stiffness: 26.1
  deform_damping_stiffness: 2.3954296985162875
  deform_elastic_stiffness: 11.11399113550867
  deform_friction_coeff: 8.885104935403207
  mass: 9.050708824606582
  scale: 49.94753606362635
params_dynamic_cotton_rag_2:
  deform_bending_stiffness: 26.1
  deform_damping_stiffness: 2.3954296985162875
  deform_elastic_stiffness: 11.11399113550867
  deform_friction_coeff: 8.885104935403207
  mass: 9.050708824606582
  scale: 49.94753606362635

 SIM===bullet_w_scale
Sim=bullet_w_scale rag=chequered_rag_0 params=[25.1         0.6496408   9.